In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from markovBike.forecast.forecast import calculate_daily_rides, get_daily_rides_with_weather, forecast_number_users
from markovBike.data_source.source import get_stations_data, database_queries
from markovBike.data_source.preprocess import preprocess_trips_data

import pandas as pd
import os

In [ ]:
n_trips = 500

trips_raw = get_stations_data(database_queries(n_trips)['trips'], verbose=True)

trips_raw['date'] = pd.to_datetime(trips_raw['starttime']).dt.date

daily_rides = calculate_daily_rides(trips_raw)

merged_df = trips_raw.merge(daily_rides, on='date')

weather_dataframe = get_daily_rides_with_weather(merged_df, os.environ.get('API_KEY'))

In [ ]:
weather_dataframe.head(3)

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numerical_features = weather_dataframe.select_dtypes(
    include=numerics).columns.drop(
        ['start_station_id', 'end_station_id', 'bikeid'])

categorical_features = weather_dataframe.select_dtypes(
    include='object').columns.drop([
        'start_station_name', 'end_station_name'
    ]).append(pd.Index(['start_station_id', 'end_station_id', 'bikeid']))

boolean_features = weather_dataframe.select_dtypes(include='bool').columns

In [ ]:
numerical_features

In [ ]:
weather_dataframe_preproc = preprocess_trips_data(
    weather_dataframe,
    index=None,
    numerical_features=numerical_features,
    categorical_features=categorical_features,
    boolean_features=boolean_features,
    drops=None,
    verbose=True)

In [ ]:
weather_dataframe_preproc = pd.DataFrame(
    weather_dataframe_preproc[0],
    columns=weather_dataframe_preproc[1]).dropna()


In [ ]:
print(list(weather_dataframe_preproc.columns))

In [ ]:
model, mae = forecast_number_users(
    weather_dataframe_preproc,
    X=['temperature', 'humidity', 'wind_speed', 'description_clear sky'],
    y='daily_n_rides',
    test_size=0.2,
    lstm_units=64,
    lstm_epochs=50,
    verbose=0)

In [ ]:
mae